In [1]:
import numpy as np
import keras
import os
from pathlib import Path

In [2]:
# Configurations
batch_size = 64  #batch size for training
epochs = 100  #number of epochs to train for.
latent_dim = 256  #latent dimensionality of the encoding space.
num_samples = 10000  #number of samples to train on.
# path to the data txt file on disk.
data_path = "./fra-eng/fra.txt"

In [3]:
# Preapre the data
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path,"r",encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) -1)]:
    input_text, target_text, _ = line.split("\t")
    # we use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text +"\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [4]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 91
Max sequence length for inputs: 14
Max sequence length for outputs: 59


In [7]:
input_token_index = dict([(char,i) for i,char in enumerate(input_characters)])
target_token_index = dict([(char,i) for i, char in enumerate(target_characters)])

In [17]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

In [19]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(10000, 14, 70)
(10000, 59, 91)
(10000, 59, 91)


In [18]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] =1.0
    encoder_input_data[i, t+1 :, input_token_index[" "]] = 1.0
    for t,char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t>0:
            # decoder_target_data will be ahead by one timestep
            # and will not include thhe start character.
            decoder_target_data[i, t-1, target_token_index[char]] = 1.0
    decoder_input_data[i, t+1:, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [21]:
# Building the model
# Define an input sequence and process it.
encoder_inputs = keras.layers.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

#we discard 'encoder_outputs' and only keep the states.
encoder_states = [state_h, state_c]

#set up the decoder, using 'encoder_states' as initial state.
decoder_inputs = keras.layers.Input(shape=(None, num_decoder_tokens))

#we set up our decoder to return full output sequences,
#and to return internal states as well. We dont use the 
#return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True,return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#define the model that will turn
#'encoder_input_data' & 'decoder_input_data' into 'decoder_target_data'
model = keras.models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [22]:
# train the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', 
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size = batch_size, epochs=epochs, validation_split=0.2)


Epoch 1/100


125/125 [==============================] - 52s 332ms/step - loss: 1.2133 - accuracy: 0.7337 - val_loss: 1.0680 - val_accuracy: 0.7146
Epoch 2/100
125/125 [==============================] - 44s 350ms/step - loss: 0.9355 - accuracy: 0.7492 - val_loss: 1.0660 - val_accuracy: 0.7155
Epoch 3/100
125/125 [==============================] - 38s 305ms/step - loss: 0.8411 - accuracy: 0.7669 - val_loss: 0.9401 - val_accuracy: 0.7355
Epoch 4/100
125/125 [==============================] - 38s 303ms/step - loss: 0.7520 - accuracy: 0.7911 - val_loss: 0.7789 - val_accuracy: 0.7782
Epoch 5/100
125/125 [==============================] - 42s 338ms/step - loss: 0.6758 - accuracy: 0.8063 - val_loss: 0.7176 - val_accuracy: 0.7929
Epoch 6/100
125/125 [==============================] - 49s 392ms/step - loss: 0.6287 - accuracy: 0.8174 - val_loss: 0.7070 - val_accuracy: 0.7940
Epoch 7/100
125/125 [==============================] - 47s 375ms/step - loss: 0.5987 - accuracy: 0.8245 - val_loss: 0.651

In [24]:
model.save("s2s_model.keras")

In [53]:
model.layers

In [54]:
model.input

[<KerasTensor: shape=(None, None, 70) dtype=float32 (created by layer 'input_3')>,
 <KerasTensor: shape=(None, None, 91) dtype=float32 (created by layer 'input_4')>]

In [55]:
'''
encoder_model = keras.models.Model(encoder_inputs, encoder_states)

decoder_state_input_h = keras.layers.Input(shape=(latent_dim,))
decoder_state_input_c = keras.layers.Input(shape=(latent_dim,))
decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)
decoder_states =[state_h, state_c]

decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = keras.models.Model([decoder_inputs] + decoder_states_inputs,
                                   [decoder_outputs] + decoder_states)
'''

'\nencoder_model = keras.models.Model(encoder_inputs, encoder_states)\n\ndecoder_state_input_h = keras.layers.Input(shape=(latent_dim,))\ndecoder_state_input_c = keras.layers.Input(shape=(latent_dim,))\ndecoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]\n\ndecoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)\ndecoder_states =[state_h, state_c]\n\ndecoder_outputs = decoder_dense(decoder_outputs)\n\ndecoder_model = keras.models.Model([decoder_inputs] + decoder_states_inputs,\n                                   [decoder_outputs] + decoder_states)\n'

In [62]:
# define sampling models
# restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s_model.keras")

encoder_inputs = model.input[0]
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.models.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]
decoder_state_input_h = keras.layers.Input(shape=(latent_dim,))
decoder_state_input_c = keras.layers.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(decoder_inputs, 
                                                         initial_state =decoder_states_inputs)

decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.models.Model([decoder_inputs] + decoder_states_inputs,
                                   [decoder_outputs] + decoder_states)

In [63]:
reverse_input_char_index = dict((i,char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [71]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq, verbose =0)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1,num_decoder_tokens))
    # populate the first character of target sequence with the start character.
    target_seq[0,0,target_token_index["\t"]] = 1.0

    # sampling loop for a batch of sequences
    #(to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value, verbose =0)
        
        # sample a token
        sampled_token_index = np.argmax(output_tokens[0,-1,:])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # exit condition: either hit max length
        # or find stop character.
        if sampled_char =='\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # update states
        states_value = [h, c]

    return decoded_sentence

In [72]:
for seq_index in range(20):
    # take one sequence (part of training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index +1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Allez-nous !

-
Input sentence: Go.
Decoded sentence: Allez-nous !

-
Input sentence: Go.
Decoded sentence: Allez-nous !

-
Input sentence: Go.
Decoded sentence: Allez-nous !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sente